In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import requests
import time
import csv
import json

In [2]:
client = Socrata("data.melbourne.vic.gov.au", None)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("4n3a-s6rn", limit=50000)
# Convert to pandas DataFrame
historic_df = pd.DataFrame.from_records(results)
historic_df

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,betweenstreet1id,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bayid,inviolation,vehiclepresent
0,17605,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,13762E,550,Loading Zone 60mins,Docklands,592,DOEPEL WAY,590,DOCKLANDS DRIVE,1048,NEWQUAY PROMENADE,2,East,6650,0,0
1,17585,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,13740E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6513,0,1
2,17562,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,13711W,NaN,NaN,Docklands,1405,WATTLE ROAD,880,LEVEN AVENUE,590,DOCKLANDS DRIVE,5,West,6483,0,1
3,17554,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,13697W,NaN,NaN,Docklands,1043,NAVIGATION DRIVE,528,COLLINS STREET,123,BOURKE STREET,5,West,6435,0,0
4,17580,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,13730E,196,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6508,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,17691,2020-03-16T12:32:34.000,2020-03-16T12:46:56.000,14,13857S,637,LZ 30Mins 7.30am -6.30PM,Docklands,164,CARAVEL LANE,1227,SAINT MANGOS LANE,36,AQUITANIA WAY,4,South,6614,0,1
49996,17730,2020-01-08T18:30:00.000,2020-01-08T18:38:55.000,8,C13504,NaN,NaN,Docklands,123,BOURKE STREET,753,HARBOUR ESPLANADE,697,GEOGRAPHE STREET,1,Centre,6102,0,1
49997,17743,2020-03-02T10:22:14.000,2020-03-02T10:23:58.000,1,C13556,196,2P MTR M-SAT 7:30-18:30,Docklands,123,BOURKE STREET,753,HARBOUR ESPLANADE,697,GEOGRAPHE STREET,1,Centre,6115,0,0
49998,17695,2020-02-21T05:59:23.000,2020-02-21T06:01:14.000,2,13861S,NaN,NaN,Docklands,164,CARAVEL LANE,1227,SAINT MANGOS LANE,36,AQUITANIA WAY,4,South,6616,0,1


In [3]:
historic_df = historic_df[["bayid", "arrivaltime", "departuretime", "durationminutes", "areaname"]]
historic_df

,bayid,arrivaltime,departuretime,durationminutes,areaname
0,6650,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,Docklands
1,6513,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,Docklands
2,6483,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,Docklands
3,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,Docklands
4,6508,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,Docklands
...,...,...,...,...,...
49995,6614,2020-03-16T12:32:34.000,2020-03-16T12:46:56.000,14,Docklands
49996,6102,2020-01-08T18:30:00.000,2020-01-08T18:38:55.000,8,Docklands
49997,6115,2020-03-02T10:22:14.000,2020-03-02T10:23:58.000,1,Docklands
49998,6616,2020-02-21T05:59:23.000,2020-02-21T06:01:14.000,2,Docklands


In [4]:
historic_df['bayid'] = historic_df['bayid'].astype(int)

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
historic_df.dtypes

bayid               int64
arrivaltime        object
departuretime      object
durationminutes    object
areaname           object
dtype: object

In [6]:
historic_df.rename(columns = {'bayid':'bay_id'}, inplace = True) 
historic_df

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,bay_id,arrivaltime,departuretime,durationminutes,areaname
0,6650,2020-01-31T15:08:32.000,2020-01-31T16:22:54.000,74,Docklands
1,6513,2020-02-26T05:43:24.000,2020-02-26T06:52:06.000,69,Docklands
2,6483,2020-01-13T22:55:52.000,2020-01-13T22:56:09.000,1,Docklands
3,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,Docklands
4,6508,2020-01-08T17:05:43.000,2020-01-08T17:40:29.000,35,Docklands
...,...,...,...,...,...
49995,6614,2020-03-16T12:32:34.000,2020-03-16T12:46:56.000,14,Docklands
49996,6102,2020-01-08T18:30:00.000,2020-01-08T18:38:55.000,8,Docklands
49997,6115,2020-03-02T10:22:14.000,2020-03-02T10:23:58.000,1,Docklands
49998,6616,2020-02-21T05:59:23.000,2020-02-21T06:01:14.000,2,Docklands


In [7]:
client = Socrata("data.melbourne.vic.gov.au", None)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("vh2v-4nfs", limit=50000)
# Convert to pandas DataFrame
live_df = pd.DataFrame.from_records(results)
live_df

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,5761,12439S,Unoccupied,"{'latitude': '-37.81193637715751', 'longitude'...",-37.81193637715751,144.97561571940406,1
1,8322,12105W,Present,"{'latitude': '-37.810063793262884', 'longitude...",-37.810063793262884,144.9834845659567,1
2,7966,15020E,Unoccupied,"{'latitude': '-37.81749471348834', 'longitude'...",-37.81749471348834,144.9819974921765,1
3,5862,C9150,Unoccupied,"{'latitude': '-37.79843097783437', 'longitude'...",-37.79843097783437,144.96501800911716,1
4,1006,951W,Present,"{'latitude': '-37.81305337978721', 'longitude'...",-37.81305337978721,144.96250770939437,1
...,...,...,...,...,...,...,...
3343,5580,12093W,Unoccupied,"{'latitude': '-37.81083727337871', 'longitude'...",-37.81083727337871,144.98335547111526,1
3344,3364,6171W,Unoccupied,"{'latitude': '-37.80426415060094', 'longitude'...",-37.80426415060094,144.94314065249569,1
3345,6390,13636E,Present,"{'latitude': '-37.82036873067143', 'longitude'...",-37.82036873067143,144.94457489244326,1
3346,3007,5068E,Present,"{'latitude': '-37.805044328852205', 'longitude...",-37.805044328852205,144.95801660850978,1


In [8]:
live_df.drop_duplicates("bay_id", inplace=True)
live_df

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,5761,12439S,Unoccupied,"{'latitude': '-37.81193637715751', 'longitude'...",-37.81193637715751,144.97561571940406,1
1,8322,12105W,Present,"{'latitude': '-37.810063793262884', 'longitude...",-37.810063793262884,144.9834845659567,1
2,7966,15020E,Unoccupied,"{'latitude': '-37.81749471348834', 'longitude'...",-37.81749471348834,144.9819974921765,1
3,5862,C9150,Unoccupied,"{'latitude': '-37.79843097783437', 'longitude'...",-37.79843097783437,144.96501800911716,1
4,1006,951W,Present,"{'latitude': '-37.81305337978721', 'longitude'...",-37.81305337978721,144.96250770939437,1
...,...,...,...,...,...,...,...
3343,5580,12093W,Unoccupied,"{'latitude': '-37.81083727337871', 'longitude'...",-37.81083727337871,144.98335547111526,1
3344,3364,6171W,Unoccupied,"{'latitude': '-37.80426415060094', 'longitude'...",-37.80426415060094,144.94314065249569,1
3345,6390,13636E,Present,"{'latitude': '-37.82036873067143', 'longitude'...",-37.82036873067143,144.94457489244326,1
3346,3007,5068E,Present,"{'latitude': '-37.805044328852205', 'longitude...",-37.805044328852205,144.95801660850978,1


In [9]:
new_df = live_df[["bay_id", "lat", "lon"]]
new_df

,bay_id,lat,lon
0,5761,-37.81193637715751,144.97561571940406
1,8322,-37.810063793262884,144.9834845659567
2,7966,-37.81749471348834,144.9819974921765
3,5862,-37.79843097783437,144.96501800911716
4,1006,-37.81305337978721,144.96250770939437
...,...,...,...
3343,5580,-37.81083727337871,144.98335547111526
3344,3364,-37.80426415060094,144.94314065249569
3345,6390,-37.82036873067143,144.94457489244326
3346,3007,-37.805044328852205,144.95801660850978


In [10]:
new_df.dtypes

bay_id    object
lat       object
lon       object
dtype: object

In [11]:
new_df['bay_id'] = new_df['bay_id'].astype(int)

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_merge_col = pd.merge(historic_df, new_df, how='right', on='bay_id')
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,Docklands,-37.81985853347056,144.9459236821079
1,6435,2020-02-03T07:30:00.000,2020-02-03T09:24:27.000,114,Docklands,-37.81985853347056,144.9459236821079
2,6435,2020-01-20T11:01:33.000,2020-01-20T11:19:02.000,18,Docklands,-37.81985853347056,144.9459236821079
3,6435,2020-02-10T16:24:33.000,2020-02-10T16:59:54.000,35,Docklands,-37.81985853347056,144.9459236821079
4,6435,2020-01-17T18:01:56.000,2020-01-17T18:09:48.000,8,Docklands,-37.81985853347056,144.9459236821079
...,...,...,...,...,...,...,...
39818,5580,NaN,NaN,NaN,NaN,-37.81083727337871,144.98335547111526
39819,3364,NaN,NaN,NaN,NaN,-37.80426415060094,144.94314065249569
39820,6390,NaN,NaN,NaN,NaN,-37.82036873067143,144.94457489244326
39821,3007,NaN,NaN,NaN,NaN,-37.805044328852205,144.95801660850978


In [13]:
df_merge_col.drop_duplicates("bay_id", inplace=True)
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,Docklands,-37.81985853347056,144.9459236821079
174,6506,2020-01-16T00:00:00.000,2020-01-16T02:17:15.000,137,Docklands,-37.81428979306522,144.93745329218785
1049,6496,2020-02-26T22:53:10.000,2020-02-26T23:08:51.000,15,Docklands,-37.81576496632796,144.93801183461784
2348,6421,2020-01-28T08:35:23.000,2020-01-28T08:40:17.000,5,Docklands,-37.82103901739355,144.94543085388358
3497,6507,2020-01-11T12:29:49.000,2020-01-11T13:34:06.000,65,Docklands,-37.81424102883436,144.93742363487377
...,...,...,...,...,...,...,...
39818,5580,NaN,NaN,NaN,NaN,-37.81083727337871,144.98335547111526
39819,3364,NaN,NaN,NaN,NaN,-37.80426415060094,144.94314065249569
39820,6390,NaN,NaN,NaN,NaN,-37.82036873067143,144.94457489244326
39821,3007,NaN,NaN,NaN,NaN,-37.805044328852205,144.95801660850978


In [14]:
df_merge_col = df_merge_col.dropna(subset = ["areaname"])
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,Docklands,-37.81985853347056,144.9459236821079
174,6506,2020-01-16T00:00:00.000,2020-01-16T02:17:15.000,137,Docklands,-37.81428979306522,144.93745329218785
1049,6496,2020-02-26T22:53:10.000,2020-02-26T23:08:51.000,15,Docklands,-37.81576496632796,144.93801183461784
2348,6421,2020-01-28T08:35:23.000,2020-01-28T08:40:17.000,5,Docklands,-37.82103901739355,144.94543085388358
3497,6507,2020-01-11T12:29:49.000,2020-01-11T13:34:06.000,65,Docklands,-37.81424102883436,144.93742363487377
...,...,...,...,...,...,...,...
36687,6699,2020-04-04T07:30:00.000,2020-04-04T07:32:37.000,2,Regency,-37.80861319837623,144.9717777829774
36688,6320,2020-01-02T21:42:21.000,2020-01-02T22:13:10.000,31,Docklands,-37.821271039627376,144.9510326336336
36689,6758,2020-03-06T00:54:31.000,2020-03-06T06:22:53.000,328,Docklands,-37.821005322182685,144.94422256740768
36690,6045,2020-01-28T04:16:35.000,2020-01-28T05:17:22.000,61,Docklands,-37.81982751532851,144.95122373557066


In [15]:
# Converting columns in the dataframe to separate tables to be saved as csv 
def convert_column_to_fk(df, column, output_dir):
    values = sorted(list(df[column].unique()))
    ids = range(1, len(values) + 1)    
    fk_df = pd.DataFrame(values, index = ids).reset_index()
    fk_df.columns = ["id", column]
    fk_df.to_csv(f"{output_dir}/{column}.csv", index = False)
    df[column] = df[column].replace(dict(zip(values, ids)))
    df = df.rename(columns = {column: f"{column}_id"})
    return df

In [16]:
df_merge_col['areaname'] = df_merge_col['areaname'].astype(str)

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
convert_column_to_fk(df_merge_col,"areaname","output")

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,bay_id,arrivaltime,departuretime,durationminutes,areaname_id,lat,lon
0,6435,2020-02-06T00:00:00.000,2020-02-06T07:30:00.000,450,1,-37.81985853347056,144.9459236821079
174,6506,2020-01-16T00:00:00.000,2020-01-16T02:17:15.000,137,1,-37.81428979306522,144.93745329218785
1049,6496,2020-02-26T22:53:10.000,2020-02-26T23:08:51.000,15,1,-37.81576496632796,144.93801183461784
2348,6421,2020-01-28T08:35:23.000,2020-01-28T08:40:17.000,5,1,-37.82103901739355,144.94543085388358
3497,6507,2020-01-11T12:29:49.000,2020-01-11T13:34:06.000,65,1,-37.81424102883436,144.93742363487377
...,...,...,...,...,...,...,...
36687,6699,2020-04-04T07:30:00.000,2020-04-04T07:32:37.000,2,5,-37.80861319837623,144.9717777829774
36688,6320,2020-01-02T21:42:21.000,2020-01-02T22:13:10.000,31,1,-37.821271039627376,144.9510326336336
36689,6758,2020-03-06T00:54:31.000,2020-03-06T06:22:53.000,328,1,-37.821005322182685,144.94422256740768
36690,6045,2020-01-28T04:16:35.000,2020-01-28T05:17:22.000,61,1,-37.81982751532851,144.95122373557066
